In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import time
import json
import os
from os import path
import io

In [12]:
#available conferences: bigtwelve, bigten, sec
#don't need to include schools.json here cause schools['json'] = schools['id']
conference = 'sec'
teamDir = "..//html//247//" + conference + "//teams//"
recruitDir = "..//html//247//" + conference + "//recruits//"
prospectDir = "..//html//247//" + conference + "//prospects//"

In [34]:
all_recruits = []

for file in os.listdir(teamDir):
    gameSoup = BeautifulSoup(open(teamDir + file, "r", encoding='windows-1252').read(), 'lxml')
    starter = "ri-page__list-item"
    team = file.split('_')[0]
    y = (file.split('_')[1]).split('.')[0]
    for x in gameSoup.find_all("li", class_="ri-page__list-item"):
        #append player to sqlinsert list
        player={}
        player_status = ""
        if (x.find("p", class_="commit-date") is not None):
            player_status = x.find("p", class_="commit-date").text
        if (player_status.strip() == 'Enrolled' or player_status.strip() == 'Signed'):
            player['school'] = team
            player['year'] = y
            #Name
            if (x.find("a", class_="ri-page__name-link") is not None):
                player['playerName'] = x.find("a", class_="ri-page__name-link").text
            #School/CityState
            if (x.find("span", class_="meta") is not None):
                locationInfoRaw = x.find("span", class_="meta")
                locationInfoList = locationInfoRaw.text.split("(")
                player['highSchool'] = (locationInfoList[0].strip())
                cityState = locationInfoList[1].split(', ')
                if (len(cityState) > 1):
                    #print(cityState)
                    state = cityState[1].strip()
                    #print(state.rstrip(')'))
                    player['city'] = (cityState[0].strip())
                    player['state'] = (state.rstrip(')'))
                else:
                    player['city'] = "None"
                    player['state'] = "None"
            #Position
            if (x.find("div", class_="position") is not None):
                player['position'] = ((x.find("div", class_="position").text).strip())
            #Height/Weight
            if (x.find("div", class_="metrics") is not None):
                heightWeight = x.find("div", class_="metrics").text.strip()
                height = (heightWeight.split(' / '))[0]
                inchHeightPre = height.split('-')
                if (inchHeightPre[0] != ''):
                    inchHeight = int(inchHeightPre[0])*12 + float(inchHeightPre[1])
                else:
                    inchHeight = '0.0'
                weight = (heightWeight.split(' / '))[1]
                if (weight == '-'):
                    weight = '0'
                player['height'] = (inchHeight)
                player['weight'] = (weight)
            
            ## We are going to get rankings data from the prospect page, but nothing else
            prospectFile = prospectDir + player['playerName'] + "_" + player['school'] + "_secondhop_" + player['year'] + ".html"
            if (path.isfile(prospectFile)):
                prospectSoup = BeautifulSoup(open(prospectFile, "r", encoding='utf8').read(), 'lxml')
                ratingsSection = prospectSoup.find_all("section", class_="rankings-section")
                count = 0
                for rating in ratingsSection:
                    if (count == 0):
                        #Composite Rating
                        if (rating.find("div", class_="rank-block") is not None):
                            player['compRating'] = (rating.find("div", class_="rank-block").text)
                        #Composite Stars
                        ratingChildren = rating.find_all("span", class_='icon-starsolid yellow')
                        i = 0
                        for child in ratingChildren:
                            i = i + 1
                        player['compStars'] = (i)
                        for y in rating.find_all("li", class_=None):
                            print(y)
                            for z in y.find_all("b", class_=None):
                                if (z == 'Natl.'):
                                    player['nationalRank'] = z.find("strong").text
                                    player['positionRank'] = z.find("strong").text
                                    player['stateRank'] = z.find("strong").text
                                else:
                                    player['positionRank'] = z.find("strong").text
                                    player['stateRank'] = z.find("strong").text
                    elif (count == 1):
                        #247 Rating
                        if (x.find("div", class_="rank-block") is not None):
                            player['247Rating'] = (x.find("div", class_="rank-block").text)
                        #Stars
                        ratingChildren = x.find_all("span", class_='icon-starsolid yellow')
                        i = 0
                        for child in ratingChildren:
                            i = i + 1
                        player['247Stars'] = (i)
                        #247 Rankings are stupid
                    count += 1
        if player:
            all_recruits.append(player)  
print(all_recruits)

<li> <b>Natl.</b> <a href="https://247sports.com/Season/2015-Football/CompositeRecruitRankings/?InstitutionGroup=HighSchool"> <strong>14</strong> </a> <a class="rank-history-link" href="https://247sports.com/PlayerSport/Daylon-Mack-at-Gladewater-44093/RecruitRankHistory/"> History </a> </li>


AttributeError: 'NoneType' object has no attribute 'text'

In [23]:
with open("..//scraped_json//247_bigtwelve.json", "w") as write_file:
    json.dump(all_recruits, write_file)

In [ ]:
#National Rank
            if (x.find("a", class_="natrank") is not None):
                player['compNationalRank'] = ((x.find("a", class_="natrank").text).strip())
            #Position Rank
            if (x.find("a", class_="posrank") is not None):
                player['compPositionRank'] = ((x.find("a", class_="posrank").text).strip())
            #State Rank
            if (x.find("a", class_="sttrank") is not None):
                player['compStateRank'] = ((x.find("a", class_="sttrank").text).strip())